In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PIL import ImageOps
import urllib3
import sys
import os
%matplotlib inline

In [2]:

def resize(image_name, dirPath):
    
    image_path = folderPath + image_name
    im = Image.open(image_path)
        
    desired_size = 128
    old_size = im.size  

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    im = im.resize(new_size, Image.ANTIALIAS)
    
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))

    delta_w = desired_size - new_size[0]
    delta_h = desired_size - new_size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    new_im = ImageOps.expand(im, padding)

    filename, file_extension = os.path.splitext(image_name)
    new_filename = filename + "_resized.jpeg"
    new_im.save(folderPath + new_filename, "JPEG")
    
    return new_filename

In [3]:

def equalize_image(image_name, dirPath): 
    
    image_path = folderPath+ image_name
    im = Image.open(image_path)
       
    im_out = ImageOps.equalize(im)
    
    filename, file_extension = os.path.splitext(image_name) 
    new_filename = filename + "_equalized.jpeg"
    im_out.save(folderPath + new_filename, "JPEG")
    
    return new_filename

In [44]:
rootDir = "/Users/barnatasa/Desktop/Màster_Bioinformàtica/TFM/lung_colon_image_set/train"
directories = ['aca', 'n', 'scc']

number_files = 0


progress = 0

for directory in directories:   
    folderPath = rootDir + '/' + directory + '/'
    filelist = os.listdir(folderPath) 
    number_files += len(filelist)
    print('Folder: {}'.format(folderPath))
    
    for fname in os.listdir(folderPath):

       
        try:
            resized_image_name = resize(fname, folderPath)
            equalized_image_name = equalize_image(resized_image_name, folderPath)
        except Exception as e:
            print('Following exception occured during the processing of {}: {}'.format(fname, str(e)))

   
        try:
            os.remove(folderPath + fname)
        except Exception as e:
            print('Exception occured when removing {} : {}'.format(fname, str(e)))
        try:
            os.remove(resized_image_name)
        except Exception as e:
            print('Exception occured when removing {} : {}'.format(resized_image_name, str(e)))

        progress += 1

        if progress % 100 == 0:
            print('Current progress : {}/{}'.format(str(progress), str(number_files)))

sized.jpeg : [Errno 2] No such file or directory: 'lungscc1548_resized.jpeg'
Exception occured when removing lungscc171_resized.jpeg : [Errno 2] No such file or directory: 'lungscc171_resized.jpeg'
Exception occured when removing lungscc1661_resized.jpeg : [Errno 2] No such file or directory: 'lungscc1661_resized.jpeg'
Exception occured when removing lungscc521_resized.jpeg : [Errno 2] No such file or directory: 'lungscc521_resized.jpeg'
Exception occured when removing lungscc1231_resized.jpeg : [Errno 2] No such file or directory: 'lungscc1231_resized.jpeg'
Exception occured when removing lungscc872_resized.jpeg : [Errno 2] No such file or directory: 'lungscc872_resized.jpeg'
Exception occured when removing lungscc937_resized.jpeg : [Errno 2] No such file or directory: 'lungscc937_resized.jpeg'
Exception occured when removing lungscc1724_resized.jpeg : [Errno 2] No such file or directory: 'lungscc1724_resized.jpeg'
Exception occured when removing lungscc1374_resized.jpeg : [Errno 2] N

In [1]:

import os
import pandas as pd
import numpy as np
np.random.seed(42)
from PIL import Image
from scipy.stats import randint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import Convolution2D, MaxPooling2D 
from keras.utils import np_utils 
from keras.utils import normalize

In [2]:
def label_encoder(key):
    label_mapping = {
        "aca" : "1",
        "n" : "2",
        "scc" : "3",
    }
    return int(label_mapping[key])

In [3]:
rootDir = "/Users/barnatasa/Desktop/Màster_Bioinformàtica/TFM/lung_colon_image_set/train"
directories = ['aca', 'n', 'scc']


df = pd.DataFrame()
category = [] 
pixel_array = [] 

for directory in directories:   
    folderPath = rootDir + '/' + directory + '/'
    print('Folder: {}'.format(folderPath))
    for fname in os.listdir(folderPath):
        if fname.endswith('resized_equalized.jpeg'):
            im = Image.open(folderPath + fname)
            im_array = np.array(im, dtype=float)
            
            category.append(label_encoder(directory))
            pixel_array.append(im_array)

pd_dict = {
    'pixel_array' : pixel_array,
    'category' : category,
}
df = pd.DataFrame(pd_dict)
df['category'] = pd.to_numeric(df['category'])
df = shuffle(df)

Folder: /Users/barnatasa/Desktop/Màster_Bioinformàtica/TFM/lung_colon_image_set/train/aca/
Folder: /Users/barnatasa/Desktop/Màster_Bioinformàtica/TFM/lung_colon_image_set/train/n/
Folder: /Users/barnatasa/Desktop/Màster_Bioinformàtica/TFM/lung_colon_image_set/train/scc/


In [4]:
print(df.shape)
df.info()
df.head()

(6000, 2)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6000 entries, 1782 to 860
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pixel_array  6000 non-null   object
 1   category     6000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 140.6+ KB


,pixel_array,category
1782,"[[[78.0, 113.0, 106.0], [95.0, 126.0, 128.0], ...",1
3917,"[[[47.0, 44.0, 27.0], [42.0, 25.0, 80.0], [165...",2
221,"[[[136.0, 127.0, 182.0], [162.0, 154.0, 203.0]...",1
2135,"[[[209.0, 194.0, 235.0], [203.0, 158.0, 189.0]...",2
5224,"[[[114.0, 126.0, 104.0], [166.0, 177.0, 163.0]...",3


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['pixel_array'].values.tolist(),
                                                    df['category'].values.tolist(),
                                                    test_size = 0.2,
                                                    random_state=42,
                                                    stratify=df['category'])
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4800, 128, 128, 3)
(1200, 128, 128, 3)
(4800,)
(1200,)


In [7]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[3], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[3], X_test.shape[1], X_test.shape[2])

In [8]:
print(X_train.shape)
print(X_test.shape)

(4800, 3, 128, 128)
(1200, 3, 128, 128)


In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = normalize(X_train)
X_test = normalize(X_test)

In [10]:
Y_train = np_utils.to_categorical(y_train, 4)[:,1:]
Y_test = np_utils.to_categorical(y_test, 4)[:,1:]
print(Y_train.shape)
for i in range(0,10):
    print('image ' + str(i+1) + ': ' + str(Y_train[i]))

(4800, 3)
image 1: [0. 0. 1.]
image 2: [0. 0. 1.]
image 3: [0. 1. 0.]
image 4: [1. 0. 0.]
image 5: [0. 0. 1.]
image 6: [1. 0. 0.]
image 7: [0. 1. 0.]
image 8: [1. 0. 0.]
image 9: [0. 0. 1.]
image 10: [0. 0. 1.]


In [49]:
model = Sequential()

In [50]:
from keras.constraints import maxnorm


model.add(Convolution2D(32, 1, 1, activation='relu', input_shape=(3,128,128)))

model.add(Convolution2D(32, 1, 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.25)) 

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_constraint=maxnorm(2)))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))

In [51]:
print(model.input_shape)
print(model.output_shape)

(None, 3, 128, 128)
(None, 3)


In [52]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [53]:
model.fit(X_train, Y_train, epochs=20, validation_split=0.1)

Epoch 1/20
135/135 [==============================] - 4s 23ms/step - loss: 1.0867 - accuracy: 0.4074 - val_loss: 0.7666 - val_accuracy: 0.7375
Epoch 2/20
135/135 [==============================] - 3s 22ms/step - loss: 0.7470 - accuracy: 0.6843 - val_loss: 0.6229 - val_accuracy: 0.7771
Epoch 3/20
135/135 [==============================] - 3s 22ms/step - loss: 0.5951 - accuracy: 0.7548 - val_loss: 0.5499 - val_accuracy: 0.7792
Epoch 4/20
135/135 [==============================] - 3s 22ms/step - loss: 0.5285 - accuracy: 0.7759 - val_loss: 0.5414 - val_accuracy: 0.7875
Epoch 5/20
135/135 [==============================] - 3s 22ms/step - loss: 0.4751 - accuracy: 0.7951 - val_loss: 0.4679 - val_accuracy: 0.7979
Epoch 6/20
135/135 [==============================] - 3s 22ms/step - loss: 0.4489 - accuracy: 0.8181 - val_loss: 0.4934 - val_accuracy: 0.7646
Epoch 7/20
135/135 [==============================] - 3s 22ms/step - loss: 0.4362 - accuracy: 0.8148 - val_loss: 0.4648 - val_accuracy: 0.7812

In [54]:
val_loss, val_acc = model.evaluate(X_test, Y_test)
print(model.metrics_names)
print(val_loss, val_acc)

38/38 [==============================] - 0s 4ms/step - loss: 0.3578 - accuracy: 0.8383
['loss', 'accuracy']
0.3578004837036133 0.8383333086967468


In [55]:
model.save('gear_classifier_4.model')

INFO:tensorflow:Assets written to: gear_classifier_4.model/assets


In [59]:
from keras.models import load_model
from sklearn import metrics
from sklearn.metrics import confusion_matrix
model = load_model('gear_classifier_4.model')
predictions = model.predict(X_test)
y_pred = (predictions > 0.5)


In [60]:
a = []

for i in y_test:
    if i == 1:
        a.append([1,0,0])
    elif i == 2:
        a.append([0,1,0])
    else:
        a.append([0,0,1])


In [61]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_pred.astype(int), a)

array([[[672,  44],
        [128, 356]],

       [[788,  34],
        [ 12, 366]],

       [[754, 121],
        [ 46, 279]]])